In [1]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Protecting Sensitive Data in Gen AI model responses

## Overview

[Sensitive Data Protection](https://cloud.google.com/security/products/sensitive-data-protection) is a fully managed service designed to discover, classify, and protect your sensitive data wherever it resides. It uses a variety of methods to identify sensitive data including regular expressions, dictionaries, and contextual elements. Once sensitive data is identified, Sensitive Data Protection (Cloud Data Loss Prevention) can take several actions to either classify, mask, encrypt, or even delete it.

Sensitive Data Protection can be accessed via Google Cloud console and used to scan data within Cloud Storage, BigQuery and other Google Cloud services. The following notebook demonstrates using the [Python Client for Cloud Data Loss Prevention](https://cloud.google.com/python/docs/reference/dlp/latest) to incorporate Sensitive Data Protection capabilities directly with Generative AI enabled applications. 

With this Python client, you define custom functions that can identify and take corrective action on sensitive data within Large Language Models (LLM) responses in real time. Throughout this notebook, you generate example text with sensitive data and run the results through custom Python functions that redact the sensitive data from Gemini 2.0 Flash model responses, so you can see this functionality in action on example data. 

After learning how to work with the Python client, you can adapt these same Python functions for Gen AI applications in your organization to protect sensitive data across your workflows.  

Notebook credit: [Jim Miller, Google](https://github.com/JimMiller-0)

### Objectives

In this lab, you learn how to use Sensitive Data Protection through the Python Client for Cloud Data Loss Prevention and explore how to identify and redact sensitive data within responses from the Gemini 2.0 Flash model.

The steps performed include:

- Installing the Python packages for Vertex AI and Cloud Data Loss Prevention (DLP) API
- Generating examples with sensitive data using Gemini 2.0 Flash model
- Defining and running Python functions to redact different types of sensitive data in Gemini 2.0 Flash model responses using the DLP API

### Costs

This tutorial uses billable components of Google Cloud:

- Vertex AI
- Sensitive Data Protection (Cloud Data Loss Prevention)

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing) and [Sensitive Data Protection](https://cloud.google.com/dlp/pricing). Use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.


## Getting started with this notebook

Below are few steps to get your environment ready including installing a few key Python packages and setting your environmental variables (project ID and region). 

Be sure to run each cell in consecutive order using the `Run` button (play arrow) at the top of this notebook. 

### Install necessary packages 

In [2]:
# Install Gen AI
!pip install --upgrade google-genai

# Install Cloud Data Loss Prevention
!pip install google-cloud-dlp --upgrade --user

  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.1.2
    Uninstalling tenacity-9.1.2:
      Successfully uninstalled tenacity-9.1.2
  Attempting uninstall: google-genai
    Found existing installation: google-genai 1.19.0
    Uninstalling google-genai-1.19.0:
      Successfully uninstalled google-genai-1.19.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [google-genai] [google-genai]


### Restart current runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which will restart the current kernel.

In [3]:
# Restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

<div class="alert alert-block alert-warning">
<b><p>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. ⚠️</p> When prompted, click OK to continue. </b>
</div>

### Set your project ID and region

In [1]:
# Set project ID and region for location
# You can find these details on the lab instruction page under Task 2
PROJECT_ID = "qwiklabs-gcp-01-2b0c5deedd62"  # @param {type:"string"}
LOCATION = "us-west4"  # @param {type:"string"}

In [2]:
# Please like share & subscribe to Techcps
# YouTube https://www.youtube.com/@techcps

print("Please like share & subscribe to Techcps https://www.youtube.com/@techcps")

Please like share & subscribe to Techcps https://www.youtube.com/@techcps


## Generate simple example text with personally identifiable information (full name) using Gemini 2.0 Flash model

The Gemini 2.0 Flash (`gemini-2.0-flash-001`) model is designed to handle natural language tasks, multi-turn text and code chat, and code generation. 

In this section, you use the the model to generate examples of text with personally identifiable information (PII) and then define a custom Python function to redact this sensitive data from the model responses.   

In [3]:
# Create the API client
from google import genai
client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)
model = "gemini-2.0-flash-001"

In [4]:
# Write a prompt that generates a simple example of personally identifiable information (full name)
prompt = f"""Who is the CEO of Google?
  """

# Run model with prompt
response_name = client.models.generate_content(
    model=model,
    contents=prompt
)

# Print response without deidentification (full name is visible)
response_name

GenerateContentResponse(
  automatic_function_calling_history=[],
  candidates=[
    Candidate(
      avg_logprobs=-0.01961918310685591,
      content=Content(
        parts=[
          Part(
            text="""The CEO of Google is Sundar Pichai.
"""
          ),
        ],
        role='model'
      ),
      finish_reason=<FinishReason.STOP: 'STOP'>
    ),
  ],
  create_time=datetime.datetime(2025, 6, 26, 3, 14, 46, 130393, tzinfo=TzInfo(UTC)),
  model_version='gemini-2.0-flash-001',
  response_id='JrtcaNn6B9im4_UP6YuE2AI',
  sdk_http_response=HttpResponse(
    headers=<dict len=9>
  ),
  usage_metadata=GenerateContentResponseUsageMetadata(
    candidates_token_count=11,
    candidates_tokens_details=[
      ModalityTokenCount(
        modality=<MediaModality.TEXT: 'TEXT'>,
        token_count=11
      ),
    ],
    prompt_token_count=9,
    prompt_tokens_details=[
      ModalityTokenCount(
        modality=<MediaModality.TEXT: 'TEXT'>,
        token_count=9
      ),
    ],
    total

## Define and run a Python function to deidentify Gemini 2.0 Flash model responses using built-in global infotypes

Sensitive Data Protection uses information types, or infoTypes, to define what it scans for. An infoType is a type of sensitive data, such as a name, telephone number, or identification number. 

In the cell below, you define a Python function that identifies and redacts that specific infoTypes that you provide as input, based on the list of built-in global infoTypes that are available in Sensitive Data Protection. Global infoTypes include general and globally applicable infoTypes such as names, date of birth, and credit card numbers. 

When you apply the function to model responses, you specify a few key built-in infoTypes to redact, such as `PERSON_NAME`, `DATE_OF_BIRTH`, and `CREDIT_CARD_NUMBER`. You can review the documentation to see the full list of [built-in infoTypes](https://cloud.google.com/sensitive-data-protection/docs/concepts-infotypes).

Run the code block below without modifications.

In [5]:
# Define function to inspect and deidentify output with Sensitive Data Protection
import google.cloud.dlp  
from typing import List 

def deidentify_with_replace_infotype(
    project: str, item: str, info_types: List[str]
) -> None:
    """Uses the Data Loss Prevention API to deidentify sensitive data in a
    string by replacing it with the info type.
    Args:
        project: The Google Cloud project id to use as a parent resource.
        item: The string to deidentify (will be treated as text).
        info_types: A list of strings representing info types to look for.
            A full list of info type categories can be fetched from the API.
    Returns:
        None; the response from the API is printed to the terminal.
    """

    # Instantiate a client
    dlp = google.cloud.dlp_v2.DlpServiceClient()

    # Convert the project id into a full resource id.
    parent = f"projects/{PROJECT_ID}"

    # Construct inspect configuration dictionary
    inspect_config = {"info_types": [{"name": info_type} for info_type in info_types]}

    # Construct deidentify configuration dictionary
    deidentify_config = {
        "info_type_transformations": {
            "transformations": [
                {"primitive_transformation": {"replace_with_info_type_config": {}}}
            ]
        }
    }

    # Call the API
    response = dlp.deidentify_content(
        request={
            "parent": parent,
            "deidentify_config": deidentify_config,
            "inspect_config": inspect_config,
            "item": {"value": item},
        }
    )

    # Print results
    print(response.item.value)

In [6]:
# Deidentify model response that includes a person's name (full name is redacted)
deidentify_with_replace_infotype(PROJECT_ID, response_name.text, ["PERSON_NAME"])

The CEO of Google is [PERSON_NAME].



## Generate and de-identify example text with more personally identifiable information (date of birth) using Gemini 2.0 Flash model

In this example, you generate an example with more personally identifiable information in the form of a medical visit log, which can include other sensitive data such date of birth.

When you run the de-identification function, you provide `PERSON_NAME` and `DATE_OF_BIRTH` as the infoTypes to redact. 

In [7]:
# Write a prompt that generates an example with more personally identifiable information (such as date of birth in a medical visit log)
prompt = f"""Generate an example medical after-visit log with faux personally identifiable information including name and date of birth
  """

# Run model with prompt
response_visitlog = client.models.generate_content(
    model=model,
    contents=prompt
)

# Print response without deidentification (full names and date of birth are visible)
response_visitlog

GenerateContentResponse(
  automatic_function_calling_history=[],
  candidates=[
    Candidate(
      avg_logprobs=-0.21497322999259091,
      content=Content(
        parts=[
          Part(
            text="""Okay, here's an example of a medical after-visit log with faux PII.  **Please remember this information is entirely fabricated and should not be used for any real-world purpose.**

**Important Disclaimer:**  Creating and using realistic-looking but fake PII can still have ethical and legal implications if used inappropriately. This example is provided for illustrative purposes only.  Do not use this information to impersonate someone, create fake accounts, or engage in any fraudulent activities.

**After-Visit Summary**

**Patient Name:**  Eleanor Vance
**Date of Birth:** 1978-03-15
**Medical Record Number:**  EV780315-42
**Date of Visit:** 2024-10-27
**Provider:** Dr. Amelia Hernandez, MD
**Clinic:**  Willow Creek Family Medicine
**123 Main Street, Anytown, CA 91234**

**Reaso

In [8]:
# Deidentify model response that includes an example medical visit log (full names and date of birth are redacted)
deidentify_with_replace_infotype(PROJECT_ID, response_visitlog.text, ["PERSON_NAME","DATE_OF_BIRTH"])

Okay, here's an example of a medical after-visit log with faux PII.  **Please remember this information is entirely fabricated and should not be used for any real-world purpose.**

**Important Disclaimer:**  Creating and using realistic-looking but fake PII can still have ethical and legal implications if used inappropriately. This example is provided for illustrative purposes only.  Do not use this information to impersonate someone, create fake accounts, or engage in any fraudulent activities.

**After-Visit Summary**

**Patient Name:**  [PERSON_NAME]
**Date of Birth:** [DATE_OF_BIRTH]
**Medical Record Number:**  EV780315-42
**Date of Visit:** 2024-10-27
**Provider:** Dr. [PERSON_NAME], MD
**Clinic:**  Willow Creek Family Medicine
**123 Main Street, Anytown, CA 91234**

**Reason for Visit:**  Annual Physical Exam

**Vitals:**

*   **Blood Pressure:** 122/78 mmHg
*   **Heart Rate:** 72 bpm
*   **Respiration Rate:** 16 breaths/min
*   **Temperature:** 98.6°F (37°C)
*   **Height:** 5'6"

## Generate example text with credit card information using Gemini 2.0 Flash model

In the previous examples, you generated example text with personally identifiable information such as full name and date of birth.

In this example, you start with generating example text with credit card information with the prompt provided below. Then, you apply what you have learned in the previous examples to run the function to redact credit card information. 

In [9]:
# Write a prompt that generates an example with a credit card number
prompt = f"""Is 4111 1111 1111 1111 an example of a credit card number?
  """

# Run model with prompt
response_creditcard = client.models.generate_content(
    model=model,
    contents=prompt
)

# Print response without deidentification (credit card number is visible)
response_creditcard

GenerateContentResponse(
  automatic_function_calling_history=[],
  candidates=[
    Candidate(
      avg_logprobs=-0.44492803835401346,
      content=Content(
        parts=[
          Part(
            text="""No, 4111 1111 1111 1111 is not a valid credit card number. Here's why:

*   **Length:** Credit card numbers typically have 13-19 digits. This number has 16 digits, so that part is okay.

*   **Issuer Identification Number (IIN):** The first few digits of a credit card number identify the card issuer. A common Visa number starts with "4", as this number does.

*   **Luhn Algorithm (Mod 10 Check):** Credit card numbers must pass the Luhn algorithm, a checksum formula used to validate identification numbers. The provided number does **not** pass the Luhn algorithm.

In summary, while the length and starting digit are within the realm of possible card numbers, the Luhn algorithm check fails, which is crucial for a valid credit card number. It's likely a test number or a randomly ge

## Test your skills using the built-in global infoType for credit card number

Now it's your turn to call the function `deidentify_with_replace_infotype` with the appropriate inputs to redact credit card numbers from model responses.

__Hint__: you can review the [global infoTypes](https://cloud.google.com/sensitive-data-protection/docs/infotypes-reference#global) in the documentation to identify the appropriate infoType for credit card numbers.

For the full solution, return to the lab instructions and expand the __Hint__ button. 

In [10]:
# Deidentify model response that includes an example credit card number (credit card number is redacted)

# ADD YOUR CODE BELOW
deidentify_with_replace_infotype(PROJECT_ID, response_creditcard.text, ["CREDIT_CARD_NUMBER"])


No, [CREDIT_CARD_NUMBER] is not a valid credit card number. Here's why:

*   **Length:** Credit card numbers typically have 13-19 digits. This number has 16 digits, so that part is okay.

*   **Issuer Identification Number (IIN):** The first few digits of a credit card number identify the card issuer. A common Visa number starts with "4", as this number does.

*   **Luhn Algorithm (Mod 10 Check):** Credit card numbers must pass the Luhn algorithm, a checksum formula used to validate identification numbers. The provided number does **not** pass the Luhn algorithm.

In summary, while the length and starting digit are within the realm of possible card numbers, the Luhn algorithm check fails, which is crucial for a valid credit card number. It's likely a test number or a randomly generated sequence.


## Redefine the Python function to block Gemini 2.0 Flash model responses based on specific infotypes for documents

In addition to its ability to scan and classify information contained within documents, Sensitive Data Protection can classify documents into multiple enterprise-specific categories. When combined with sensitive data inspection, this classification can be useful for document risk assessment, policy enforcement, and similar use cases.

In this section, you redefine the the original function to take advantage of this classification functionality and use it to block output for two specific [document infoTypes](https://cloud.google.com/sensitive-data-protection/docs/infotypes-reference#documents): source code and patents.

In the code block below for the function, notice the new code lines after `# Add conditional return for document infoTypes for source code and patent`. 

Run the code block below without modifications.

In [11]:
# Redefine original function to inspect and deidentify output with Sensitive Data Protection
import google.cloud.dlp  
from typing import List 

def deidentify_with_replace_infotype(
    project: str, item: str, info_types: List[str]
) -> None:
    """Uses the Data Loss Prevention API to deidentify sensitive data in a
    string by replacing it with the info type.
    Args:
        project: The Google Cloud project id to use as a parent resource.
        item: The string to deidentify (will be treated as text).
        info_types: A list of strings representing info types to look for.
            A full list of info type categories can be fetched from the API.
    Returns:
        None; the response from the API is printed to the terminal.
    """

    # Instantiate a client
    dlp = google.cloud.dlp_v2.DlpServiceClient()

    # Convert the project id into a full resource id.
    parent = f"projects/{PROJECT_ID}"

    # Construct inspect configuration dictionary
    inspect_config = {"info_types": [{"name": info_type} for info_type in info_types]}

    # Construct deidentify configuration dictionary
    deidentify_config = {
        "info_type_transformations": {
            "transformations": [
                {"primitive_transformation": {"replace_with_info_type_config": {}}}
            ]
        }
    }

    # Call the API for deidentify
    response = dlp.deidentify_content(
        request={
            "parent": parent,
            "deidentify_config": deidentify_config,
            "inspect_config": inspect_config,
            "item": {"value": item},
        }
    )

    return_payload = response.item.value
    
    # Add conditional return to block responses containing document infoTypes for source code and patent
    info_types = ["DOCUMENT_TYPE/R&D/SOURCE_CODE","DOCUMENT_TYPE/R&D/PATENT"]
    inspect_config = {"info_types": [{"name": info_type} for info_type in info_types]}

    response = dlp.inspect_content(
        request={
            "parent": parent,
            "inspect_config": inspect_config,
            "item": {"value": item},
        }
    )

    if response.result.findings:
        for finding in response.result.findings:
            if finding.info_type.name == "DOCUMENT_TYPE/R&D/SOURCE_CODE":
                return_payload = '[Blocked due to category: Source Code]'
            elif finding.info_type.name == "DOCUMENT_TYPE/R&D/PATENT":
                return_payload = '[Blocked due to category: Patent Related]'
                
    # Print results
    print(return_payload)

## Generate an example with source code using Gemini 2.0 Flash model and block results

In the previous examples, you generated example text with personally identifiable information.

In this example, you generate examples with document infoTypes including source code and patent information. Then, you apply what you have learned in the previous examples to run the function to block responses based on these document infoTypes. Also, in this example, when generating the response, you keep the [temperature](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/prompts/adjust-parameter-values#temperature) to `0`, so that the generated output is invariable. 

In [12]:
# Create prompt that generates an example of Java code
prompt = f"""Show me an example of Java code
  """

# Run model with prompt
response_sourcecode = client.models.generate_content(
    model=model,
    contents=prompt,
    config={
        "temperature": 0,
    	}
)

# Print response without blocking it (code is visible)
response_sourcecode

GenerateContentResponse(
  automatic_function_calling_history=[],
  candidates=[
    Candidate(
      avg_logprobs=-0.06178712632921007,
      content=Content(
        parts=[
          Part(
            text="""```java
public class HelloWorld {

    public static void main(String[] args) {
        // This is a comment.  It's ignored by the compiler.

        // Print "Hello, World!" to the console.
        System.out.println("Hello, World!");

        // Example of a variable declaration and assignment
        String message = "This is a message stored in a variable.";
        System.out.println(message);

        // Example of an integer variable and arithmetic
        int number1 = 10;
        int number2 = 5;
        int sum = number1 + number2;
        System.out.println("The sum of " + number1 + " and " + number2 + " is: " + sum);

        // Example of a conditional statement (if-else)
        if (sum > 12) {
            System.out.println("The sum is greater than 12.");
       

In [13]:
# Block model response that include source code (response is not available)
# Notice that the infoType that you request is a different infoType
# Results are still blocked because the model response is identified contain code
deidentify_with_replace_infotype(PROJECT_ID, response_sourcecode.text, ["EMAIL_ADDRESS"])

[Blocked due to category: Source Code]


## Test your skills using the built-in document infoType for patents

Now it's your turn to call the function `deidentify_with_replace_infotype` with the appropriate inputs to block patent information in model responses.

__Hint__: review the previous two cells for generating an example with source code and calling the function, and then modify both to block the model response because it contains patent information.

For the full solution, return to the lab instructions and expand the __Hint__ button. 

In [15]:
# Create prompt that generates example patent

prompt = f"""Show me an example patent

"""

# Run model with prompt

# Name the output as response_patent

response_patent = client.models.generate_content(model=model, contents=prompt, config={"temperature": 0,})

# Print response without blocking it (patent information provided)

response_patent

GenerateContentResponse(
  automatic_function_calling_history=[],
  candidates=[
    Candidate(
      avg_logprobs=-0.22733503896978838,
      content=Content(
        parts=[
          Part(
            text="""Okay, let's look at a real patent.  I'll provide you with a link to a patent on Google Patents, along with a brief explanation of what it covers.  I'll choose a relatively simple and understandable one.

**Patent:** US 8,086,984 B2 - "Method and apparatus for providing a user interface for a mobile device"

**Link:** [https://patents.google.com/patent/US8086984B2/en](https://patents.google.com/patent/US8086984B2/en)

**What it Covers (Simplified):**

This patent, granted to Apple Inc., generally describes a method and apparatus for providing a user interface on a mobile device (like a smartphone or tablet) that allows users to easily access and interact with applications and other functions.  Specifically, it focuses on:

*   **A Dock:**  A persistent area (often at the bottom 

In [16]:
# Block model response that includes patent information (patent information not provided)

# ADD YOUR CODE BELOW
deidentify_with_replace_infotype(PROJECT_ID, response_patent.text, ["EMAIL_ADDRESS"])

[Blocked due to category: Patent Related]
